In [1]:
!pip install stanza
import stanza
import sqlite3

In [5]:
con = sqlite3.connect("proyectoPura.db")
cur = con.cursor()

#We create a query to extract all the titles from one column
#from different tables OBRAS, COMPLEMENTOS Y SECCIONES:
query = """
SELECT Título_Obra FROM OBRAS
UNION ALL
SELECT Título_Complemento FROM COMPLEMENTOS
UNION ALL
SELECT Título_Sección FROM SECCIONES_PUBLICADAS
"""
#we reunite them:
cur.execute(query)
titles = cur.fetchall()
#print(titles)

#they return us titles in a list of tuples,
#but we need a list of strings to create easier a .txt with all of them as rows
titles_list = []
for title in titles:
  if title[0]:  #some rows are NULL, we avoid them
    titles_list.append(title[0])
#print(titles_list)

with open("content/old_nadege/ALL_TITLES.txt", "w", encoding="utf-8") as f: ##N: eso para tus ojos, no es necesario en tu workflow 
  ##dado que tienes tu lista titles_list sobre la cual puedes trabajar (y aplicarle processing_1 definida más abajo) 
  for elemento in titles_list:
    f.write(elemento + "\n")

con.close()

In [7]:
def processing_1(text, nlp_model):
  doc = nlp_model(text)
  return [(word.text, word.lemma, word.upos, word.head, word.deprel) for sent in doc.sentences for word in sent.words]

nlp_es = stanza.Pipeline(lang="es", processors="tokenize,mwt,pos,lemma,ner,depparse")

processed_titles_es = [processing_1(tit, nlp_es) for tit in titles_list]
path = "content/old_nadege/ALL_TITLES.txt"

with open(path, "r", encoding="utf-8") as f:
  rows = f.readlines()

processed_titles_es = [processing_1(row, nlp_es) for row in rows]

processed_titles_es

with open("content/old_nadege/_ES_PROCESSING_PERTITLE.txt", "w", encoding="utf-8") as f: ##igual
  for element in processed_titles_es:
    f.write(str(element) + "\n")

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: es (Spanish):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| depparse  | combined_charlm   |
| ner       | conll02           |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [12]:
# Before applying NER, first cleaning of each title
# (only specific symbols that affect NER, such us "[...]", "-", ".") %%

import regex
def clean_text(text):
  cleaned_text = re.sub(r'\s*"\s*|\s*-\s*|\s*\[\.\.\.\]\s*', ' ', text)
  cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
  return cleaned_text

def AreThereSubtitles(title):
  cortado_re = '"([^"]+)\[\.\.\.\]\s*"'
  es_cortado = regex.findall(cortado_re, title)
  if es_cortado and len(es_cortado)>1:
    subtitulos_re = '"([^"]+)(\[\.\.\.\])?\s*"(\s*-\s*"([^"]+)(\[\.\.\.\])?\s*")+'
    subtitulos = regex.search(subtitulos_re, title)
    if subtitulos:
        print(title)
        print(subtitulos.groups())
        print('\n')
    
  

path = "content/old_nadege/ALL_TITLES.txt"
with open(path, "r", encoding="utf-8") as f:
  rows = f.readlines()
  cleaned_rows = [clean_text(row) for row in rows]
  for row in rows:
    AreThereSubtitles(row)

#print(cleaned_rows)

with open("content/old_nadege/CLEAN_ALL_TITLES.txt", "w", encoding="utf-8") as f:
    for element in cleaned_rows:
        f.write(str(element) + "\n")

"El matrimonio de la Princesa real de Inglaterra [...]"- "Hace algunos años que un banquero muy rico [...]"-"Varios periódicos franceses y del extranjero [...]"

('El matrimonio de la Princesa real de Inglaterra [...]', None, '-"Varios periódicos franceses y del extranjero [...]"', 'Varios periódicos franceses y del extranjero [...]', None)


"El viaje de SS. AA. [...]"-"El sábado a la una [...]"-"El príncipe Mirza-Ally-Kan-Bahadoor [...]"-"Habláse mucho en los círculos aristocrático de un matrimonio [...]"

('El viaje de SS. AA. [...]', None, '-"Habláse mucho en los círculos aristocrático de un matrimonio [...]"', 'Habláse mucho en los círculos aristocrático de un matrimonio [...]', None)


"El lunes 5 del corriente se ha inaugurado [...]"-"Vamos a contar una historia ocurrida en estos días de Pascua"-"Cuando Ismail, generalísimo del ejército otomano [...]"

('El lunes 5 del corriente se ha inaugurado [...]', None, '-"Cuando Ismail, generalísimo del ejército otomano [...]"', 'Cuando I

<>:11: SyntaxWarning: invalid escape sequence '\['
<>:14: SyntaxWarning: invalid escape sequence '\['
<>:11: SyntaxWarning: invalid escape sequence '\['
<>:14: SyntaxWarning: invalid escape sequence '\['
/tmp/ipykernel_327886/884667257.py:11: SyntaxWarning: invalid escape sequence '\['
  cortado_re = '"([^"]+)\[\.\.\.\]\s*"'
/tmp/ipykernel_327886/884667257.py:14: SyntaxWarning: invalid escape sequence '\['
  subtitulos_re = '"([^"]+)(\[\.\.\.\])?\s*"(\s*-\s*"([^"]+)(\[\.\.\.\])?\s*")+'


In [11]:
#let's iterate on cleaned_rows through NER function

def ner1(text, nlp_model):
  doc = nlp_model(text)
  return [(ent.text, ent.type) for ent in doc.entities]

nlp_es = stanza.Pipeline(lang="es", processors="tokenize,mwt,pos,lemma,ner")

cleaned_entities_ES = [ner1(title, nlp_es) for title in cleaned_rows]

print(cleaned_entities_ES)

with open("content/CLEANED_ENTITIES_ES.txt", "w", encoding="utf-8") as f:
    for element in cleaned_entities_ES:
        f.write(str(element) + "\n")

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: es (Spanish):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| ner       | conll02           |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


[[('Sr. D. R. P.', 'MISC')], [('Fragmentos', 'MISC')], [('Desvarío', 'MISC')], [('Soneto', 'PER'), ('Quintana', 'PER')], [('Balada', 'PER')], [('Luna', 'MISC'), ('Mocquet', 'PER')], [('señora M…', 'PER')], [], [], [('Luna', 'MISC')], [('Don B. M.', 'PER')], [('Manzanares', 'LOC')], [('Luna', 'MISC')], [('de la Rosa de Alejandra', 'PER')], [], [('Luna', 'MISC')], [('Sr. D. Agustín Esteban Collantes', 'MISC')], [('Margarita Aurora de Wilson', 'PER'), ('Serrano', 'PER')], [('I', 'PER')], [('Margarita', 'PER')], [], [('Señora doña M. Del Pilar Sinués de Marco', 'PER')], [('II', 'MISC')], [('Margarita', 'PER')], [('Margarita Aurora de Wilson', 'PER'), ('Serrano', 'PER')], [('María', 'PER')], [('Alteza Real', 'PER'), ('Príncipe de Asturias', 'PER')], [('Margarita', 'PER')], [], [('Soneto', 'PER')], [('SS. MM. II', 'PER')], [('Princesa', 'PER'), ('Inglaterra', 'LOC')], [('Poesía', 'MISC'), ('Oda', 'PER')], [('Alejandro Dumas', 'PER')], [('Soneto', 'PER')], [], [('SS. AA', 'ORG'), ('príncipe M

In [12]:
# and now, we also want to process (tok, lemma, mkw, pos) the NER string,
# without applying depparse %%
def processing_2(text, nlp_model):
  doc = nlp_model(text)
  return [(word.text, word.lemma, word.upos) for sent in doc.sentences for word in sent.words]

clean_processed_entities_ES = []

for entity_list in cleaned_entities_ES:
  row_result = []
  for entity in entity_list:
    text = entity[0]
    tok = processing_2(text, nlp_es)
    row_result.append([entity, tok])

  clean_processed_entities_ES.append(row_result) #IMPORTANT TO NOT INDENT THIS LINE, OTHERWISE IT DOES NOT KEEP EMPTY ROWS(THAT IS, WITHOUT ENTITY)

with open("content/CLEAN_MERGED_TITLE_NER.txt", "w", encoding="utf-8") as f:
    for element in clean_processed_entities_ES:
        f.write(str(element) + "\n")

clean_processed_entities_ES[:10]

[[[('Sr. D. R. P.', 'MISC'),
   [('Sr', 'sr', 'PROPN'),
    ('.', '.', 'PUNCT'),
    ('D.', 'd.', 'PROPN'),
    ('R.', 'R.', 'PROPN'),
    ('P.', 'p.', 'PROPN')]]],
 [[('Fragmentos', 'MISC'), [('Fragmentos', 'fragmentos', 'NOUN')]]],
 [[('Desvarío', 'MISC'), [('Desvarío', 'desvarío', 'NOUN')]]],
 [[('Soneto', 'PER'), [('Soneto', 'soneto', 'NOUN')]],
  [('Quintana', 'PER'), [('Quintana', 'Quintana', 'PROPN')]]],
 [[('Balada', 'PER'), [('Balada', 'Balada', 'PROPN')]]],
 [[('Luna', 'MISC'), [('Luna', 'luna', 'PROPN')]],
  [('Mocquet', 'PER'), [('Mocquet', 'Mocquet', 'PROPN')]]],
 [[('señora M…', 'PER'),
   [('señora', 'señora', 'NOUN'), ('M…', 'm…', 'PROPN')]]],
 [],
 [],
 [[('Luna', 'MISC'), [('Luna', 'luna', 'PROPN')]]]]

In [13]:
# let's locate the entire entity and its processing [[(),[()]]] where their
# strings are located within each title %%
import re
#clean_processed_entities_ES
cleaned_rows_copy = cleaned_rows.copy()

for i, titulo in enumerate(clean_processed_entities_ES):
  for componentes in titulo:
    for tuple_item in componentes:
      entity_str = tuple_item[0][0]
      if entity_str in cleaned_rows_copy[i]:
        replacement_str = f"{tuple_item}"
        cleaned_rows_copy[i] = re.sub(rf'\b{entity_str}\b', replacement_str, cleaned_rows_copy[i])

print("\n".join(cleaned_rows_copy))

Al [('Sr', 'sr', 'PROPN'), ('.', '.', 'PUNCT'), ('D.', 'd.', 'PROPN'), ('R.', 'R.', 'PROPN'), ('P.', 'p.', 'PROPN')]. D. R. P. Enviándole mis obras
La mujer piadosa ([('Fragmentos', 'fragmentos', 'NOUN')])
[('Desvarío', 'desvarío', 'NOUN')]
[('Soneto', 'soneto', 'NOUN')]. ¡Gloria y pobreza! El inmortal [('Quintana', 'Quintana', 'PROPN')]...
[('Balada', 'Balada', 'PROPN')]. La despedida de una hija
Un viaje a la [('Luna', 'luna', 'PROPN')]. Prólogo: La pesadilla de [('Mocquet', 'Mocquet', 'PROPN')]
En el álbum de la [('señora', 'señora', 'NOUN'), ('M…', 'm…', 'PROPN')] M…
La primavera
La perejilera
Un viaje a la [('Luna', 'luna', 'PROPN')]. (Continuación)
Al señor [('Don', 'Don', 'PROPN'), ('B.', 'B.', 'PROPN'), ('M.', 'M.', 'PROPN')] B. M.
A orillas del [('Manzanares', 'manzanares', 'PROPN')]. La súplica
Un viaje a la [('Luna', 'luna', 'PROPN')]. (Continuación)
Fragmentos [('de', 'de', 'ADP'), ('la', 'el', 'DET'), ('Rosa', 'Rosa', 'PROPN'), ('de', 'de', 'ADP'), ('Alejandra', 'Alejandra